## Choroplethmapbox animation

We  define an animated Choroplethmapbox, where each frame consists in plotting only two counties from Switzerland.

In [ ]:
import numpy as np
import json
import plotly.graph_objs as go

In [ ]:
import urllib.request

def read_geojson(url):
    with urllib.request.urlopen(url) as url:
        jdata = json.loads(url.read().decode())
    return jdata 

In [ ]:
swiss_url = 'https://raw.githubusercontent.com/empet/Datasets/master/swiss-cantons.geojson'
jdata = read_geojson(swiss_url)

In [ ]:
import pandas as pd

data_url = "https://raw.githubusercontent.com/empet/Datasets/master/Swiss-synthetic-data.csv"
df = pd.read_csv(data_url)
df.head()

In [ ]:
len(jdata['features']), len(df)

Define a list of geojson type dicts (its elements would correspond to geojson files for different countries in a continent, when
                                    the animation is intended for that case):

In [ ]:
geojs =[]
for k in range(0, 26, 2):
    geojs.append(dict(type='FeatureCollection',
                features = jdata['features'][k:k+2]))

In [ ]:
geojs[5].keys()

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df[:2], 
                           geojson=geojs[0], 
                           featureidkey='properties.id',
                           locations='canton-id',
                           color='2018',
                           color_continuous_scale  ='matter_r',      
                           zoom=6.25,
                           center={'lat': 46.8181877 , 'lon':8.2275124 },
                           mapbox_style='carto-positron')

                  
fig.update(frames = fig.frames, layout = fig.layout)
fig.update_layout(title_text='Choroplethmapbox animation',
                  title_x=0.5,
                  width=900, height=500,
                  coloraxis_cmin=min(df['2018']), 
                  coloraxis_cmax=max(df['2018']),
                  coloraxis_colorbar_thickness=25);

Below we are selecting from the df a new dataframe consisting in only  two rows and one column from the general dataframe, df:

In [ ]:
k =3
df.iloc[k:k+2, [1]]

In [ ]:
df.iloc[k:k+2, [1]]['canton-id']

In [ ]:
frames = [go.Frame(
              data=[go.Choroplethmapbox(geojson=geojs[k//2], 
                                        locations=df.iloc[k:k+2, [1]]['canton-id'], # dataframe consisting in rows k:k+2 from df['canton-id'], 
                                        z=df.iloc[k:k+2, [2]]['2018'],  # rows k:k+2 from df['2018']
                                       )],
              name=f'fr{k//2}') for k in range(0, len(df), 2)]
fig.update(frames=frames);

In [ ]:
f_duration =350  #frame duration
fig.update_layout(updatemenus=[dict(type='buttons', 
                                y=0,
                                x=1.25,
                                active=0,
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=f_duration, 
                                                                    redraw=True),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])])],
                 sliders =  [dict(steps= [dict(method= 'animate',
                              args= [[ f'fr{k}'],
                                    dict(mode= 'immediate',
                                         frame= dict( duration=f_duration, redraw= True ),
                                         fromcurrent=True,
                                         transition=dict( duration= 0))],
                                         label=f"fr{k}") for k in range(len(frames))], 
                              minorticklen=0,
                              x=0,
                              len=1)]);
fig.show()

In [ ]:
import chart_studio.plotly as py
py.plot(fig, filename='choroplethmbx-anim')

**Remarks**:

**1.** This method of decomposing the initial geojson file into many geojson type dicts has the advantage that requires less 
data to be involved in the animation. 
Another method could be to set `geojson=jdata` in the definition of `fig= px.choropleth_mapbox()`,
and pass to each frame data   only  `locations` and `z`. In this case each frame inherits
the other attributes set in `fig.data[0]`, i.e.
for each frame is used the entire jdata file. Obviously this method is not recommended for large geojson files read as jdata.


**2.**  If `fig.data[0]` is defined directly as an instance of the class `go.Choroplethmapbox`, not via `px.choroplethmapbox()`, then to ensure the correct mapping of  z-values in each frame, to the associated colorscale, the attributes `zmin`, `zmax` must be set, as can be seen below (note that for `go.Choroplethmapbox` we have `zmin`, `zmax` as attributes setting the least and the biggest value to be mapped to a colorscale, while using `px.choropleth_mapbox()', the same effect has setting
coloraxis['cmin'], coloraxis['cmax'], and these differences of names  can be a bit confusing.

Alternative definition of fig, defined above by `px.choropleth_mapbox()`:

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=geojs[0], 
                                    locations=df.iloc[0:2 , [1]]['canton-id'],
                                    z=df.iloc[0:2 , [2]]['2018'], 
                                    featureidkey='properties.id',
                                    colorscale='matter_r',
                                    colorbar_thickness=25,
                                    zmin=min(df['2018']),
                                    zmax=max(df['2018']), 
                                    marker_line_width=1))


fig.update_layout(title_text = 'Choroplethmapbox animation',
                  title_x=0.5,
                  width=800, height=500,
                  mapbox=dict(style='carto-positron',
                              zoom=6.25, 
                              center = {"lat": 46.8181877 , "lon":8.2275124 },
                              ))
